# **VIDEO GENERATION AGENT**

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
from pydantic import BaseModel
import json
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Optional

load_dotenv()
if os.environ.get('GEMINI_API_KEY'):
    print("The key loaded successfully")
else:
    raise ValueError("ERROR LOADING THE KEY")

The key loaded successfully


# **GENERATING TRENDING TOPICS**

In [28]:
def generate_trending_topic():
    prompt = """You are a viral content strategist with deep expertise in TikTok, Instagram Reels, and YouTube Shorts trends.

Your task: Generate ONE unique, highly specific trending video topic that is currently viral or has high virality potential.

Requirements:
- Pick from diverse categories: tech, fitness, food hacks, fashion, DIY, travel, pets, gaming, productivity, finance tips, relationships, mental health, home decor, beauty, comedy skits, etc.
- Be SPECIFIC (not "cooking tips" but "3-second egg peeling hack that actually works")
- Avoid generic/overused topics like morning routines, coffee aesthetics, or productivity setups
- Think unexpected, niche, or contrarian angles that stop the scroll

Output ONLY the topic idea in one sentence. No explanation."""
    
    client = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.95)
    response = client.invoke(prompt)
    return response.content

# **PROMPTS**

In [29]:
def generate_trending_topic():
    prompt = """You are a viral content strategist with deep expertise in TikTok, Instagram Reels, and YouTube Shorts trends.

Your task: Generate ONE unique, highly specific trending video topic that is currently viral or has high virality potential.

Requirements:
- Pick from diverse categories: tech, fitness, food hacks, fashion, DIY, travel, pets, gaming, productivity, finance tips, relationships, mental health, home decor, beauty, comedy skits, etc.
- Be SPECIFIC (not "cooking tips" but "3-second egg peeling hack that actually works")
- Avoid generic/overused topics like morning routines, coffee aesthetics, or productivity setups
- Think unexpected, niche, or contrarian angles that stop the scroll

Output ONLY the topic idea in one sentence. No explanation."""
    
    client = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.95)
    response = client.invoke(prompt)
    return response.content

SYSTEM_PROMPT = """
You are an expert AI Cinematographer and Sound Designer. Your role is to take a creative strategy and translate it into precise technical instructions for Google Veo 3.1.

Instructions for Scene Generation:
Camera Movement: Always specify a camera motion (e.g., 'dolly-in', 'slow orbit', 'low-angle tilt').
Cinematic Lighting: Use professional lighting terms (e.g., 'volumetric fog', 'rim lighting', 'soft-box diffusion').
Soundscape Logic: Describe sounds that match the visual actions for Veo's native audio engine.

Output (Strict JSON):
{{
  "veo_prompt": "A detailed, 1-paragraph cinematic description of the scene including camera movement and lighting.",
  "motion_bucket_id": 85,
  "audio_description": "A specific description of the ambient and foley sounds.",
  "video_metadata": {{
    "resolution": "1080p",
    "aspect_ratio": "9:16",
    "fps": 24
  }}
}}
"""

USER_PROMPT = generate_trending_topic()
print(f"📌 Trending Topic: {USER_PROMPT}")

📌 Trending Topic: The hidden 'developer mode' hack on your smart TV that instantly removes all ads and bloatware without jailbreaking.


# **PYDANTIC CLASSES AND FUNCTION**

In [30]:


class VideoExecutorStrategy(BaseModel):
    video_prompt: str = Field(
        ..., 
        description="Detailed cinematic description for Veo 3.1 including camera movement and lighting."
    )
    motion_bucket_id: int = Field(
        default=85, 
        ge=1, le=255, 
        description="Motion intensity: 1 for static, 255 for extreme movement."
    )
    audio_prompt: str = Field(
        ..., 
        description="Description of the native audio soundscape (ambient sounds, music, foley)."
    )
    overlay_text: str = Field(
        ..., 
        description="The hook/headline text to be displayed on the video."
    )
    aspect_ratio: str = Field(
        default="9:16", 
        description="The frame dimensions (e.g., 9:16 for vertical, 16:9 for wide)."
    )
    video_title: str = Field(..., description="Optimized title for the video post.")
    tags: List[str] = Field(default_factory=list, description="List of 5 relevant SEO hashtags.")

    class Config:
        schema_extra = {
            "example": {
                "video_prompt": "A slow-motion tracking shot of a neon-lit futuristic garden...",
                "motion_bucket_id": 120,
                "audio_prompt": "Soft electronic hum with the sound of wind chimes.",
                "overlay_text": "Gardening in 2077",
                "aspect_ratio": "9:16",
                "video_title": "Future Garden Aesthetics",
                "tags": ["#future", "#aiart", "#gardening"]
            }
        }


def log_to_daily_jsonl(data_object, folder="logs"):
    if not os.path.exists(folder):
        os.makedirs(folder)
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = os.path.join(folder, f"{date_str}_pinterest.jsonl")
    if hasattr(data_object, 'model_dump'):
        entry = data_object.model_dump()
    else:
        entry = data_object.dict()
    entry["logged_at"] = datetime.now().isoformat()
    with open(filename, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")
    return filename

C:\Users\souga\AppData\Local\Temp\ipykernel_18788\4230415579.py:1: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class VideoExecutorStrategy(BaseModel):


# **OUTPUT**

In [31]:
dynamic_prompt = ChatPromptTemplate.from_messages([
    ('system',SYSTEM_PROMPT),
    ('human','{topic}')
])

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.7)
structured_model = model.with_structured_output(VideoExecutorStrategy)
chain = dynamic_prompt | structured_model

try:
    print("Generating Strategy JSON...")
    result = chain.invoke({'topic': USER_PROMPT})

    strategy_dict = result.model_dump() if hasattr(result, 'model_dump') else result.dict()
    with open('current_strategy.json', 'w', encoding='utf-8') as f:
        json.dump(strategy_dict, f, indent=2)
    print("✅ Saved to current_strategy.json")

    log_file = log_to_daily_jsonl(result, folder="logs")
    print(f"✅ Success! Strategy saved to {log_file}")
    print("\n" + "="*50)
    print("📹 GENERATED VIDEO STRATEGY:")
    print("="*50)
    print(f"🎬 Title: {result.video_title}")
    print(f"📝 Overlay: {result.overlay_text}")
    print(f"🎥 Prompt: {result.video_prompt[:200]}...")
    print(f"🔊 Audio: {result.audio_prompt[:100]}...")
    print(f"📐 Aspect: {result.aspect_ratio} | Motion: {result.motion_bucket_id}")
    print(f"🏷️  Tags: {', '.join(result.tags)}")
    
    
except Exception as e:
    print(f"❌ Error: {e}")

Generating Strategy JSON...
✅ Saved to current_strategy.json
✅ Success! Strategy saved to logs\2026-02-05_pinterest.jsonl

📹 GENERATED VIDEO STRATEGY:
🎬 Title: Unlock Your Smart TV's Hidden Developer Mode: Remove All Ads & Bloatware!
📝 Overlay: THE SECRET TV HACK: NO ADS, NO BLOATWARE!
🎥 Prompt: The camera executes a slow, deliberate dolly-in, starting from a medium shot of a sleek, modern smart TV displaying a typical home screen cluttered with app icons and small ad banners. The ambient lig...
🔊 Audio: A low, almost imperceptible hum of the smart TV permeates the quiet room. Subtle, crisp clicks of th...
📐 Aspect: 9:16 | Motion: 85
🏷️  Tags: #SmartTVHack, #NoAds, #DeveloperMode, #TechTips, #TVHacks
